# Introduction
In this module, we will discuss issues related to accessing data via web services. Modern web sites and applications use Application Programming Interfaces (API) to retrieve and store data in a web environment such as a browser. Most of these transactions happen behind the scenes, but, because they occur in a web environment, they are accessible through any traditional internet channel. In this module we will look at web service in the context of Financial Modeling Prep's (FMP) API. Though much of this module is specific to FMP, the process is common to many other web services.

# Web Services
For most web services, you must register with the service before you can access their data. In the following section, I illustrate how to create an FMP developer account. Then, I show how to connect to the FMP API to download data and answer questions.
## Creating an FMP Developer Account
https://site.financialmodelingprep.com/register
<br><img src="http://thislondonhouse.com/Jupyter/Images/12-FMP-01.png?1" width="50%" /><br>
Register with your loyola email address.
<br><img src="http://thislondonhouse.com/Jupyter/Images/12-FMP-02.png?1" width="50%" /><br>
After you are registered, navigate to your dashboard to access your API key. This is the token that your Python app will use to authenticate with FMP’s servers. If you do not have an API key, you will need to generate them. Copy this key and paste it into your python file (see below for instructions).


## Connecting to FMP’s API via Python
In general, there are two common approaches to connecting to an API. The first is to use http requests to pull in data from API resources and the second is to use wrapper libraries to facilitate pulling data from the API. Accessing the API directly via Python is often a more reliable method for accessing data because wrapper libraries can become outdated quickly if not actively maintained. So, while there are some wrapper libraries for accessing FMP, I will be illustrating direct API access via python.

### HTTP Requests
Most APIs simply allow external users access to source data via web pages. These web pages use URLs and query string parameters to request specific data. Facilitate these connections, we will use the Requests library (more info here: https://requests.kennethreitz.org/en/master/).

In [ ]:
!pip install Requests

This approach requires that we directly access API resources, so we need a reference guide which will outline the available resources and the parameters that each method requires and/or allows. For FMP, this reference is here: https://site.financialmodelingprep.com/developer/docs.

In [ ]:
import requests
import urllib
import pprint
import statistics

You should notice, on the documentation, the 'endpoint' is a URL and some aspects of that URL are standard across all API resource while others vary depending on the data request.
<br><img src="http://thislondonhouse.com/Jupyter/Images/12-FMP-03.png?1" width="50%" /><br>
The getEnpoint function below handles access to the API by reformatting input parameters to match the API endpoints. The response is then downloaded and returned as a dictionary which you can then use to extract data.

In [ ]:
def getEndpoint(endpoint, version, parameters):
    baseUrl = f"https://financialmodelingprep.com/api/{version}"

    endpointUrl = f"{baseUrl}/{endpoint}"

    parameters['apikey'] = "< API KEY HERE >"
    headers = {}
    payload = {}

    print("Getting Endpoint: " + endpointUrl + "?" + urllib.parse.urlencode(parameters))
    response = requests.request("GET", endpointUrl, headers=headers, data=payload, params=parameters)
    response_data = response.json()
    return response_data


In [ ]:
resource = "profile"
symbol = "GOOG"
endpoint = f"{resource}/{symbol}"
parameters = {}

In the code above, I set some parameters that will allow me to access company data for a choose ticker symbol. The 'resource' refers to the API data endpoint that will respond to our request. Documentation on this endpoint is available here: https://site.financialmodelingprep.com/developer/docs#company-profile-company-information

'symbol' refers to the ticker symbol of the company for which we are requesting data. The 'endpoint' variable restructures the resource and symbol into a format that the 'getEndpoint' function will use to make the API call. The 'parameters' variable is an empty dictionary which may or may not have values that the getEnpoint function will use to define the data request. All four variables are needed each time you want to make an API call.

In [ ]:
company_data = getEndpoint(endpoint, "v3", parameters)

The code above requests the enpoint and saves the results of the request in the 'company_data' variable. This variable is usually a dictionary, but it may sometimes be a list of dictionaries. Therefore, it may be helpful to pprint the results before trying to extract data.

In [ ]:
print(type(company_data))
pprint.pprint(company_data)
print(company_data)

In this situation, the result is a list of dictionaries with only one element. Therefore, to access the company profile data, we would want to reference the first element

In [ ]:
print(company_data[0]['description'])

This may be cumbersome. So, if the data structure happens to be nested within another structure, you can create a new variable that points directly to your chosen data structure.

In [ ]:
goog_data = company_data[0]
pprint.pprint(goog_data)

In [ ]:
goog_description = goog_data['description']

In [ ]:
print("Google Description: " + goog_description)

You would then reference other data elements in a similar way.

In [ ]:
print(goog_data['ipoDate'])

In [ ]:
print(goog_data['zip'])

In [ ]:
print(goog_data['companyName'])

In [ ]:
print(goog_data['price'])

In [ ]:
print(goog_data['industry'])

Any data elements you want to reserve, you can save as variables.

In [ ]:
print(goog_data['range'])
price_range = goog_data['range']

You could then modify this variable to find the high and low.

In [ ]:
price_range_list = price_range.split("-")
price_high = float(price_range_list[1])
price_low = float(price_range_list[0])
print("High:", price_high, "Low:", price_low)

There are several endpoints available in Financial Modeling Prep. If you wanted to download income statement data, you would simply edit the symbol, resource, and parameters variables to reflect a new data request. In the following example, we leave the symbol unchanged because we want the income statement for Alphabet Inc., but we modify the other values because we want to download their annual income statements.

In [ ]:
resource = "income-statement"
endpoint = f"{resource}/{symbol}"
parameters = {'period': 'annual'}
income_data = getEndpoint(endpoint, "v3", parameters)

It is always good practice to inspect the results with pprint.

In [ ]:
pprint.pprint(income_data)

In this case, the result is a list of dictionaries with the income statements ordered from newest to oldest. So, the first element in the list will have data from the most recent income statement.

In [ ]:
pprint.pprint(income_data[0])

If you wanted to see the revenue for each year, you could traverse the list of dictionaries and pull the relevant data from each element in the list.

In [ ]:
revenues = []
for income_statement in income_data:
    print(f"Revenues as of {income_statement['date']}: {income_statement['revenue']}")
    revenues.append(income_statement['revenue'])

In [ ]:
print(revenues)

In [ ]:
print(statistics.mean(revenues))

# Exercise
Pick a ticker and approximate the number of shares outstanding (shares outstanding = market cap / price)

In [ ]:
# Step 1...

# Step 2...